# Customer Event (Touch Point) Prediction
_**Predict a customer's future event or touchpoint using LSTM based on the past events customer participated in.**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)
1. [Predict](#Predict)




## Background
This notebook illustrates an LSTM based approach to predict customer’s future events based on the past events customer participated in.  Notebook is based on the session "A novel adoption of LSTM in customer touchpoint prediction" from Strata 2018 conference proceedings. (https://conferences.oreilly.com/artificial-intelligence/ai-ca-2018/public/schedule/detail/68831)

The notebook uses Keras, a popular well-documented open source deep learning for implementing the LSTM model. To use Keras on Amazon SageMaker, we use the built-in Tensorflow environment that includes support for Keras.  This not only simplifies the development process, it also allows you to use standard Amazon SageMaker features like script mode.  The notebook also shows how to run the same Keras code on Amazon SageMaker that you run on your local machine, using script mode.

We will follow this sequence in this notebook

1. Examine and analyze the data
2. Execute the python script that builds and trains the LSTM model using Keras. 
3. Train and deploy using Amazon SageMaker's 'local' mode
4. Train on a training cluster of machine learning instances managed by Amazon SageMaker
5. Deploy the trained model
6. Execute inferences against the deployed model

## Setup

Let's start by importing the Python libraries we'll need.

In [ ]:
import sagemaker
import boto3
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sagemaker.tensorflow import TensorFlow

Next specify : 
1. The IAM role arn used to give training and hosting access to your data. For this notebook, this is the same role you associated with this notebook instance.
2. The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.  For this notebook, we will use the default S3 bucket associated with the sagemaker session.


In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

s3_bucket = sess.default_bucket()
print("s3_bucket is : ", s3_bucket)

## Data

Our data consists of a sequence of customer events and the trained model will predict the next sequence of the customer events. 

In this particular example, sequence of the customer events is the names of the TV channels viewed.  Let's say a company is running advertisements for a specific product or a new store location in these channels.  From the customer touchpoints, that capture their TV viewing behavior, the goal is to predict the next sequence of touchpoints.  The predicted sequence of touchpoints may include the next sequence of channels the customer will view and customer conversion behavior.  Conversion of the customer could mean that the customer visited the new store location or purchased a product being advertised on the TV channels. This specific event of interest is represented by the word 'visit' in our data set.

<img src="../images/CustomerTouchPpoints.png">

Data is available in the "data.txt" file that accompanies this notebook.  It has been downloaded from https://github.com/shinchan75034/LSTM_TouchPoint. 



#### Read in the data from data.txt

In [ ]:
data_file="data.txt"

with open(data_file, 'r') as f:
    lines = f.read().split('\n')
    data = np.array(lines)
#print("Data : ", data)

#### Examine the data by viewing the first few elements of the data. 
Data is expected to have the structure of 
    **customer_id, sequence of input events and sequence of target events** each seperated by a **tab** character 


In [ ]:
##Show the first few lines of the data, to verify the data structure 
data[:10]

#### Split data into training (80%) and test (20%) data sets
Training data set is split into training and validation within the python script customer_event_prediction_lstm_keras_model.py

In [ ]:
##Split data into training (80%), test (20%) data sets

train_dat, test_dat = train_test_split(data,test_size=0.2)

#Verify training and test data set sizes
print("\nTraining Data Size ", len(train_dat))
print("Test Data Size ", len(test_dat))
        
#Convert training and test data sets to lists
train_lines = list(train_dat)
test_lines = list(test_dat)

#### Define utility methods to create corpus, split lines into input & target and encode the data

In [ ]:
#Create the corpus dictionary
def create_corpus_dict(word_list):
  token_index = dict(
    [(word, i) for i, word in enumerate(word_list)])
  return token_index

#Split a given line into input and target
def split_input_and_target(line_list):
    input_texts = []
    target_texts = []
    
    try:

        for line in line_list:
            _, input_text, target_text = line.split('\t')
            # We use "tab" as the "start sequence" character
            # for the targets, and "\n" as "end sequence" character.
            target_text = '<start>' + " " + target_text + " " + '<stop>' 
            input_texts.append(input_text)
            target_texts.append(target_text)
            
    except:
      pass
    
    return input_texts, target_texts

## Method to encode data
def encode_data(input_texts,target_texts, input_vocab, target_vocab, input_corpus, target_corpus) :
    
    #Get the array/list length/counts
    # input and target may have different vocab and different token count.
    input_vocab = sorted(list(input_vocab))
    target_vocab = sorted(list(target_vocab))
    num_encoder_tokens = len(input_vocab)
    num_decoder_tokens = len(target_vocab)
    max_encoder_seq_length = max([len(txt.split()) for txt in input_texts]) # number of words in each string.  Use max length to make all sequences same size.
    max_decoder_seq_length = max([len(txt.split()) for txt in target_texts])
    
    #Create zero encoded/decoder arrays of correct size
    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
    decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
    
    #Now update the encoded/decoded arrays with 1.
    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, word in enumerate(input_text.split()):
            encoder_input_data[i, t, input_corpus[word]] = 1.
        for t, word in enumerate(target_text.split()):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t, target_corpus[word]] = 1.
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_corpus[word]] = 1.
    
    return encoder_input_data, decoder_input_data, decoder_target_data

#### Define utility methods to upload to and read from S3 bucket.

In [ ]:
s3 = boto3.resource('s3') 

##Save ndarray to file, upload to S3
def upload_ndarray_to_s3(encoder_input_data, s3_prefix):
    local_file = 'encoded_data.npy'
    np.save(local_file, encoder_input_data) 
    s3.Bucket(s3_bucket).upload_file(local_file, s3_prefix)
    
    
### Read ndarray from S3
def read_ndarray_from_s3(s3_prefix):
    local_file_downloaded = 'downloaded_encoder_data.npy'
    s3.Bucket(s3_bucket).download_file(s3_prefix, local_file_downloaded)
    downloaded_encoder_input_data = np.load(local_file_downloaded)
    return downloaded_encoder_input_data

In [ ]:
# Set up all data to build a corpus
input_texts = []
target_texts = []
input_words = set()
target_words = set()

for line in lines:
      try:
        #print ("line :", line)
        #Split each line into input text and target text.  Ignore the user_id (???)
        _, input_text, target_text = line.split("\t")
        #print("input_text :", input_text, " output text : ", target_text)


        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        #Update target text to include <start> and <stop> tokens
        target_text = '<start>' + " " + target_text + " " + '<stop>'   


        #Append input_texts and target_texts
        input_texts.append(input_text)
        target_texts.append(target_text)

        #Split the input_text and target_text into words and populate the input_words and target_words
        for word in input_text.split():
            if word not in input_words:
                input_words.add(word)
        for word in target_text.split():
            if word not in target_words:
                target_words.add(word)
      except:
        pass

print("Number of input words ", len(input_words))
print("Number of target words ", len(target_words))  #Should be two more than input words, since we added <start> and <stop>
    
    
#Build the vocabulary.  Here it is simply union of the input and target words.
vocab = list(set(input_words).union(set(target_words)))
print("Vocab size ", len(vocab))
    
corpus_dict = create_corpus_dict(vocab)
print("corpus_dict size ", len(corpus_dict))
    
# split each set of lines into input and target separately.
train_input_texts, train_target_texts  = split_input_and_target(train_lines)
#validation_input_texts, validation_target_texts  = split_input_and_target(validation_lines)
test_input_texts, test_target_texts  = split_input_and_target(test_lines)

In [ ]:
##Encode training data and persist to S3.  This encoded data is used for training.
train_encoder_input_data, train_decoder_input_data, train_decoder_target_data = encode_data(train_input_texts,train_target_texts, vocab,vocab, corpus_dict, corpus_dict)

upload_ndarray_to_s3(train_encoder_input_data, "train/train_encoder_input_data.npy")
upload_ndarray_to_s3(train_decoder_input_data, "train/train_decoder_input_data.npy")
upload_ndarray_to_s3(train_decoder_target_data, "train/train_decoder_target_data.npy")

In [ ]:
train_encoder_input_data

In [ ]:
##Encode test data and persist to S3.  This encoded data is later on used for making inferences against the deployed model.
test_encoder_input_data, test_decoder_input_data, test_decoder_target_data = encode_data(test_input_texts,test_target_texts, vocab,vocab, corpus_dict, corpus_dict)

upload_ndarray_to_s3(test_encoder_input_data, "test/test_encoder_input_data.npy")
upload_ndarray_to_s3(test_decoder_input_data, "test/test_decoder_input_data.npy")
upload_ndarray_to_s3(test_decoder_target_data, "test/test_decoder_target_data.npy")

#### Define utility method to predict

In [ ]:
##Use the predictor passed in for making predictions on the test data at 'prediction_index'

stop_word_list = ['<start>', '<stop>']

def predict(predictor, prediction_index):
    
    #print("encode input data type : " , type(test_encoder_input_data[prediction_index]), "encode input data size : " , len(test_encoder_input_data[prediction_index ]))
    predictions_from_model = predictor.predict({'encoder_input_data' : test_encoder_input_data[prediction_index], 
                                          'decoder_input_data' : test_decoder_input_data[prediction_index]})
    
    output_tokens = np.asarray(predictions_from_model['predictions'])
    integer_list = output_tokens.argmax(axis=2)

    # Reassign variables for convenience
    input_token_index = corpus_dict
    #target_token_index = corpus_dict
    
    # Reverse-lookup token index to decode sequences back to something readable.
    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
    #reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

    translated_array = np.vectorize(reverse_input_char_index.get)(integer_list) 
  
    translated_list = translated_array.tolist()
    
    predicted_sequence = translated_list[0]
    
    ##Remove <start>, <stop> tokens from the predicted sequence
    predicted_sequence_cleaned = [item for item in predicted_sequence if item not in stop_word_list]

    return predicted_sequence_cleaned 

## Train

To train the LSTM model, we follow this sequence to demonstrate multiple training options available :

    1. Execute a python script right from this notebook.  The python script builds and trains the LSTM model using Keras.  This step allows you to verify that the python script will execute without any errors.
    
    2. Using the same python script as the entry point, we will then train (and deploy) using Amazon SageMaker 'local' mode. This step uses the "Script" mode with a prebuilt Tensorflow container launched on this very notebook instance.  This allows you to test the proper execution of the python script with Amazon SageMaker's prebuilt TensorFlow container, without the overhead of launching additional compute instances.
       
    3. Finally, we will train on a cluster of ML instances managed by Amazon SageMaker, thus creating a full scale training job.  This step is used to scale out the training job, with larger data sets and distributed training.


#### Examine the python script

In [ ]:
!cat customer_event_prediction_lstm_keras_model.py

#### Execute the python script
Verify that the python script will execute without any errors.

In [ ]:
#Delete this directory the model is already persisted in this directory.
!rm -rf /tmp/model/1

In [ ]:
!python customer_event_prediction_lstm_keras_model.py --epochs 2 --batchsize 64 --modeldir '/tmp' --s3_bucket $s3_bucket 

#### Train using the "local" mode
In this mode, training happens on this notebook instance itself.  A 
Tensorflow container is launched, the python script is executed in the container and the model is persisted to the S3 bucket.
Verify proper execution of the python script with Amazon SageMaker's prebuilt TensorFlow container, without the overhead of launching additional compute instances.

In [ ]:
#Create a Tensorflow Estimator
tf_estimator_local = TensorFlow(entry_point='customer_event_prediction_lstm_keras_model.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={ 'epochs': 1, 
                                           'batch-size': 256,
                                            'learning-rate': 0.01,
                                           's3_bucket': s3_bucket }
                         )

In [ ]:
#Call fit on the estimator to kick off training.
tf_estimator_local.fit() 

In [ ]:
#Deploy the predictor locally
tf_predictor_local = tf_estimator_local.deploy(initial_instance_count=1,
                         instance_type='local')

In [ ]:
#Invoke the predict method with the local predictor and the index of the test data items to get predictions for.
predict (tf_predictor_local, 10) 

#### Train on a training cluster
In this mode, training happens on new ML instances launced and managed by SageMaker.  Notice that the only parameter that is different is train_instance_type.  Instead of "local", we specify the type of the instance to be launched

In [ ]:
#Create a Tensorflow Estimator
tf_estimator_on_cluster = TensorFlow(entry_point='customer_event_prediction_lstm_keras_model.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.m5.xlarge',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 20,
                              'batch-size': 256,
                              'learning-rate': 0.01,
                              's3_bucket': s3_bucket}
                         )

In [ ]:
#Call fit on the estimator to kick off training. This takes approximately 13 minutes
tf_estimator_on_cluster.fit()

## Host
Next deploy the model.  This step hosts the model on ML instance(s) managed by Amazon Sagemaker and return an "Endpoint" that will be used to run inference against in the "Predict" step.

In [ ]:
## Deploy the model trained on the cluster managed by Amazon SageMaker
## This takes approximately 9 minutes
tf_endpoint_name = 'customer-event-prediction-lstm'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

tf_predictor = tf_estimator_on_cluster.deploy(initial_instance_count=1,
                         instance_type='ml.c5.large',        
                         endpoint_name=tf_endpoint_name)

## Predict

Now let's use the deployed model for predictions.  Use the test_encoder_input_data and test_decoder_input_data to make the predictions.  Once predictions are done, calculate the confusion matrix and other model metrics

In [ ]:
##Predict output for a single element (represented by the index) in the text target using the hosted predictor.
predicted_sequence = predict(tf_predictor,0)
print("predicted sequence ", predicted_sequence)

In [ ]:
## Now predict for multiple elements in the text target 
all_predictions = []
for i in range(0,len(test_encoder_input_data.tolist())):
    predicted_sequence = predict(tf_predictor, i)
    all_predictions.append(predicted_sequence)
        
print("Total number of predictions ", len(all_predictions))        
        

In [ ]:
## Calculate true/false positives/negative
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

for i in range(0,len(test_target_texts)):
    if ("visit" in all_predictions[i] and "visit" in test_target_texts[i]):
        true_positives = true_positives + 1
    elif ("visit" in all_predictions[i] and "visit" not in test_target_texts[i]):   
        false_positives = false_positives + 1
    elif ("visit" not in all_predictions[i] and "visit" not in test_target_texts[i]):   
        true_negatives = true_negatives + 1 
    elif ("visit" not in all_predictions[i] and "visit" in test_target_texts[i]):   
        false_negatives = false_negatives + 1     
        
print("true_positives : ", true_positives)  
print("false_positives : ", false_positives) 
print("true_negatives : ", true_negatives) 
print("false_negatives : ", false_negatives)


In [ ]:
#Calculate Recall, Precision, F1 and show confusion matrix

Recall = true_positives / (true_positives + false_negatives)
Precision = true_positives / (true_positives + false_positives)
F1 = 2*(Recall*Precision)/(Recall + Precision)

print("Model Metrics ")
print("\tRecall : ", Recall)  
print("\tPrecision : ", Precision) 
print("\tF1 : ", F1) 


print("\n\n========== Confusion Matrix ==========")

print("\t\tVisit\tNo Visit\n")
print("Visit\t\t", true_positives, "\t", false_positives, "\n")
print("NoVisit\t\t", false_negatives, "\t", true_negatives,)

While we can see a reasonable recall and precision with this model, there is definitely room for improvement. To improve the model metrics, consider :

1. Increasing the training dataset volume.
2. Experimenting with hyperparameter tuning to identify the right combinations of hyperparameters (epochs, learning rate, batch_size etc).




## Delete Endpoint

Run the cell below to delete the endpoint once you are done.¶
Note that till the endpoint is deleted, you are incurring costs.

In [ ]:
tf_predictor.delete_endpoint()